In [1]:
from eventregistry import *
from collections import defaultdict
from collections import Counter 
import json
from tqdm import tqdm
import unidecode

In [15]:
# API_KEY = '830dbc9b-6086-4657-8185-da9becdcdf3d'
API_KEY = '3d0ea6ef-97b8-4eb7-8315-c817bf1a1688' # PAID subscription

In [16]:
er = EventRegistry(apiKey = API_KEY)

using user provided API key for making requests
Event Registry host: http://eventregistry.org
Text analytics host: http://analytics.eventregistry.org


In [3]:
locations = ['Venezuela', 'France', 'Hong Kong', 'Iran']
sources = {
    'us': ['nytimes.com', 'cnn.com', 'voanews.com', 'foxnews.com'], 
    'china': ['globaltimes.cn', 'scmp.com', 'hongkongfp.com', 'chinadaily.com.cn'], # china news ecns.cn, shine.cn
    'france': ['france24.com', 'afp.com'],
    'iran': ['iran-daily.com', 'tehrantimes.com', 'ifpnews.com'],
    'venezuela': ['entornointeligente.com', 'laht.com'],
    'russia': ['sputniknews.com', 'rt.com', 'themoscowtimes.com', 'meduza.io'], # therussophile.org has 2.3M articles
    'qatar': ['aljazeera.com']
}
dates = {
    'Venezuela': {'date_start': '2019-01-01', 'date_end': '2019-11-30'},
    'France': {'date_start': '2018-11-01', 'date_end': '2019-03-31'},
    'Hong Kong': {'date_start': '2019-06-01', 'date_end': '2019-12-31'},
    'Iran': {'date_start': '2019-11-01', 'date_end': '2020-02-29'}
}
date_start = '2018-11-01'
date_end = '2020-02-29'
protest_key_words = [
    'protest', 'protesters', 'protester',
    'demonstration', 'demonstrations', 'demonstrators', 'demonstrator'
    'clash', 'clashes',
    'riot', 'riots',
    'barricades', 'barricade',
    'tear gas',
    'unrest', 'unrests',
    'revolt',
    'upheaval',
    'uproar',
    'manifestation', 'manifestations'
]
# Country, major cities, political figures (president...), specific names, adjectives (ex: 'french')
# protest_specific_key_words = {
#     'Venezuela': ['venezuela', 'south america', 'latin america'],
#     'France': ['france', 'paris', 'marseilles', 'lyon', '', 'yellow', 'vests', 'yellow vests', 'macron'],
#     'Hong Kong': ['hong kong', 'china'],
#     'Iran':['iran', 'tehran']
# }
protest_specific_key_words = {
'Venezuela':['maduro', 'guaido', 'venezuela', 'caracas', 'venezuelan', 'juan', 'venezuelans', 'nicolas', 'lopez', 'chavez', 'cabello', 'leopoldo', 'diosdado', 'hyperinflation', 'latin', 'shortages', 'cucuta', 'hugo', 'padrino'],
'France':['paris', 'macron', 'yellow', 'french', 'france', 'champs', 'elysees', 'vest', 'vests', 'arc', 'triomphe', 'emmanuel', 'avenue', 'christophe', 'castaner', 'jackets', 'philippe', 'elysee', 'marseille', 'michel', 'edouard', 'lyon'],
'Hong Kong':['hong', 'kong', 'china', 'lam', 'chinese', 'extradition', 'beijing', 'mainland', 'carrie', 'wong', 'long', 'yuen', 'cheung', 'chan', 'communist', 'xi', 'kowloon', 'xinjiang', 'kongers', 'chinese'],
'Iran':['iran', 'iranian', 'tehran', 'soleimani', 'khamenei', 'irgc', 'ali', 'ayatollah', 'fadavi', 'drone', 'iranians', 'irbil', 'erbil', 'rouhani', 'mousavi', 'khuzestan', 'persian', 'zarif']
}

In [18]:
stats_dict = {'nytimes.com': 8221,
 'cnn.com': 2818,
 'voanews.com': 2511,
 'foxnews.com': 5012,
 'globaltimes.cn': 350,
 'scmp.com': 5568,
 'hongkongfp.com': 1013,
 'france24.com': 4412,
 'afp.com': 5578,
 'iran-daily.com': 1996,
 'tehrantimes.com': 650,
 'ifpnews.com': 281,
 'entornointeligente.com': 2095,
 'laht.com': 2244,
 'sputniknews.com': 5085,
 'rt.com': 2298,
 'themoscowtimes.com': 371,
 'novayagazeta.ru':0,
 'meduza.io':0,
 'aljazeera.com': 4117,
 'chinadaily.com.cn':0}
stats_dict

{'nytimes.com': 8221,
 'cnn.com': 2818,
 'voanews.com': 2511,
 'foxnews.com': 5012,
 'globaltimes.cn': 350,
 'scmp.com': 5568,
 'hongkongfp.com': 1013,
 'france24.com': 4412,
 'afp.com': 5578,
 'iran-daily.com': 1996,
 'tehrantimes.com': 650,
 'ifpnews.com': 281,
 'entornointeligente.com': 2095,
 'laht.com': 2244,
 'sputniknews.com': 5085,
 'rt.com': 2298,
 'themoscowtimes.com': 371,
 'novayagazeta.ru': 0,
 'meduza.io': 0,
 'aljazeera.com': 4117,
 'chinadaily.com.cn': 0}

In [19]:
def query_articles_iter(er, source, start, end):
    print('{} {} {} expected results: {}'.format(source, start, end, stats_dict[source]))
    q = QueryArticlesIter(keywords=QueryItems.OR(protest_key_words),
            dateStart=start,
            dateEnd=end,
            sourceUri=er.getSourceUri(source),
            lang=["eng"])
    query_articles = []
    for article in tqdm(q.execQuery(er, sortBy = "rel", maxItems = 10000)):
        query_articles.append(article)
    return query_articles

# QUERY ARTICLES
# articles = defaultdict(dict)
# for source_origin in sources:
#     for source in sources[source_origin]:
#         if source not in ['nytimes.com', 'cnn.com', 'voanews.com', 'foxnews.com', 'globaltimes.cn']:
#             articles[source] = query_articles_iter(er, source, date_start, date_end)
#         # Checkpoint after each query.
#         with open('data_collection/articles_{}.json'.format(source), 'w') as article_file:
#             article_file.write(json.dumps(articles[source]))
articles = defaultdict(dict)
for source in  ['chinadaily.com.cn']:
    articles[source] = query_articles_iter(er, source, date_start, date_end)
    # Checkpoint after each query.
    with open('data_collection/articles_{}.json'.format(source), 'w') as article_file:
        article_file.write(json.dumps(articles[source]))

chinadaily.com.cn 2018-11-01 2020-02-29 expected results: 0


3652it [00:42, 86.02it/s] 


In [7]:
import json
with open('articles (copy).json') as article_file:
    articles = (json.loads(article_file.read()))

In [8]:
articles.keys()

dict_keys(['nytimes.com', 'cnn.com', 'voanews.com', 'foxnews.com', 'globaltimes.cn'])

In [9]:
for source in articles:
    with open('data_collection/articles_{}.json'.format(source), 'w') as article_file:
        article_file.write(json.dumps(articles[source]))

In [22]:
def preprocess_text(text, split=False):
    # lowercase
    text = text.lower()
    # remove accents
    text = unidecode.unidecode(text)
    if split:
        text = text.split()
    return text

def get_article_protest_counts(text, key_words_dict):
    # Input should be cleaned from accents and lowercased.
    word_counter = Counter(preprocess_text(text, split=True))
    kw_counter = Counter()
    protest_kw_counter = Counter()
    for protest in key_words_dict:
        for kw in key_words_dict[protest]:
            kw_count = word_counter[kw]
            kw_counter[kw] += kw_count
            protest_kw_counter[protest] += kw_count
    return kw_counter, protest_kw_counter

def get_protest_label(protest_kw_counter):
    key, value = protest_kw_counter.most_common(1)[0]
    if value > 2:
        return key
    else:
        return None
    
def remove_duplicates(articles):
    # Keeps most recent.
    articles_sorted = sorted(articles, key=lambda x: x['date']+'-'+x['time'], reverse=True)
    article_bodies_so_far = set()
    duplicate_count = 0
    for article in articles_sorted:
        processed_text = preprocess_text(article['body'])
        if processed_text in article_bodies_so_far:
            articles.remove(article)
            duplicate_count += 1
        article_bodies_so_far.add(processed_text)
    return articles, duplicate_count
                       
def filter_dates_protest(result, protest_kw_counter):
    for protest in protest_kw_counter:
        if result['date'] < dates[protest]['date_start'] or result['date'] > dates[protest]['date_end']:
            del protest_kw_counter['protest']
    return protest_kw_counter

articles_by_protest = defaultdict(dict)
for source_origin in ['']:#sources:
    for source in ['chinadaily.com.cn']:#sources[source_origin]:
        with open('data_collection/articles_{}.json'.format(source)) as in_file:
            results = json.loads(in_file.read())
            results, duplicates = remove_duplicates(results)
            nones = 0
            for i in range(len(results)):
                kw_counter, protest_kw_counter = get_article_protest_counts(results[i]['body'], 
                                                                            protest_specific_key_words)
                protest_kw_counter = filter_dates_protest(results[i], protest_kw_counter)
                results[i]['kw_counter'] = kw_counter
                results[i]['protest_kw_counter'] = protest_kw_counter
                label =  get_protest_label(protest_kw_counter)
                results[i]['protest'] = label
                if label == None:
                    nones += 1
            print('{} duplicates {} new count {} no useful protest {}'.format(source, duplicates, len(results), nones))
        with open('data_collection_clean/articles_{}.json'.format(source), 'w') as out_file:
            out_file.write(json.dumps(results))            

chinadaily.com.cn duplicates 17 new count 3635 no useful protest 995


In [26]:
article_list = []
protest_counter = Counter()
protest_counter_by_source = defaultdict(Counter)
for source_origin in sources:
    for source in sources[source_origin]:
        with open('data_collection_clean/articles_{}.json'.format(source)) as in_file:
            articles = json.loads(in_file.read())
            for article in articles:
                if article['protest'] is not None:
                    article_list.append(article)
                    protest_counter[article['protest']] += 1
                    protest_counter_by_source[source][article['protest']] += 1

In [27]:
protest_counter

Counter({'Hong Kong': 14822, 'France': 3577, 'Iran': 2907, 'Venezuela': 1967})

In [28]:
protest_counter_by_source

defaultdict(collections.Counter,
            {'nytimes.com': Counter({'Hong Kong': 1823,
                      'France': 417,
                      'Iran': 264,
                      'Venezuela': 248}),
             'cnn.com': Counter({'Hong Kong': 603,
                      'Iran': 103,
                      'France': 62,
                      'Venezuela': 76}),
             'voanews.com': Counter({'Hong Kong': 455,
                      'Iran': 232,
                      'France': 104,
                      'Venezuela': 55}),
             'foxnews.com': Counter({'Hong Kong': 498,
                      'France': 186,
                      'Iran': 230,
                      'Venezuela': 218}),
             'globaltimes.cn': Counter({'France': 5,
                      'Hong Kong': 300,
                      'Venezuela': 1,
                      'Iran': 4}),
             'scmp.com': Counter({'Hong Kong': 4292,
                      'France': 99,
                      'Iran': 55,
        

In [29]:
with open('articles_all.json', 'w') as out_file:
    out_file.write(json.dumps(article_list))

In [4]:
articles_list_all = []
for source_origin in sources:
    for source in sources[source_origin]:
        with open('data_collection/articles_{}.json'.format(source)) as in_file:
            articles_list_all += json.loads(in_file.read())
print(len(articles_list_all))
            

58677


In [6]:
with open('articles_all_no_filter.json', 'w') as out_file:
    out_file.write(json.dumps(articles_list_all))

In [15]:
def query_articles(er, source, start, end):
    print('{} {} {}'.format(source, start, end))
    q = QueryArticles(
        keywords=QueryItems.OR(protest_key_words),
        dateStart=start,
        dateEnd=end,
        sourceUri=er.getSourceUri(source),
        lang=["eng"]
    )
    q.setRequestedResult(RequestArticlesInfo(sortBy="rel"))
    res = er.execQuery(q)
    return res

In [16]:
# QUERY ARTICLES
articles = defaultdict(dict)
for source_origin in sources:
    for source in sources[source_origin]:
        articles[source] = query_articles(er, source, date_start, date_end)
        # Checkpoint after each query.
        with open('articles.json', 'w') as article_file:
            article_file.write(json.dumps(articles))

nytimes.com 2018-11-01 2020-02-29
cnn.com 2018-11-01 2020-02-29


KeyboardInterrupt: 

In [89]:
# PRINT INFO
num_articles = 0
num_articles_by_source = defaultdict(int)
for source in articles:
    num_articles+=len(articles[source]['articles']['results'])
    num_articles_by_source[source] += len(articles[source]['articles']['results'])
    print('{}: {}'.format( source, len(articles[source]['articles']['results'])))
        
print(num_articles)
print(dict(num_articles_by_source))

nytimes.com: 100
cnn.com: 100
voanews.com: 100
foxnews.com: 100
globaltimes.cn: 100
scmp.com: 100
hongkongfp.com: 100
france24.com: 100
afp.com: 100
iran-daily.com: 100
tehrantimes.com: 100
ifpnews.com: 100
entornointeligente.com: 100
laht.com: 100
therussophile.org: 100
sputniknews.com: 100
rt.com: 100
themoscowtimes.com: 100
aljazeera.com: 100
1900
{'nytimes.com': 100, 'cnn.com': 100, 'voanews.com': 100, 'foxnews.com': 100, 'globaltimes.cn': 100, 'scmp.com': 100, 'hongkongfp.com': 100, 'france24.com': 100, 'afp.com': 100, 'iran-daily.com': 100, 'tehrantimes.com': 100, 'ifpnews.com': 100, 'entornointeligente.com': 100, 'laht.com': 100, 'therussophile.org': 100, 'sputniknews.com': 100, 'rt.com': 100, 'themoscowtimes.com': 100, 'aljazeera.com': 100}


In [43]:
for loc in sources:
    for source in sources[loc]:
        print(er.getSourceUri(source))

nytimes.com
cnn.com
voanews.com
globaltimes.cn
scmp.com
hongkongfp.com
france24.com
afp.com
iran-daily.com
tehrantimes.com
ifpnews.com
entornointeligente.com
laht.com


In [31]:
for loc in locations:
    print(er.getLocationUri(loc))

http://en.wikipedia.org/wiki/Venezuela
http://en.wikipedia.org/wiki/France
http://en.wikipedia.org/wiki/Hong_Kong
http://en.wikipedia.org/wiki/Iran


In [68]:
res_test = query_articles(er, 'France', 'france24.com', dates['France']['date_start'], dates['France']['date_end'])

France france24.com 2018-11-01 2019-03-31


In [69]:
res_test['articles']

{'articles': {'page': 1,
  'totalResults': 653,
  'pages': 7,
  'results': [{'uri': '1002819988',
    'lang': 'eng',
    'isDuplicate': False,
    'date': '2018-11-25',
    'time': '09:22:00',
    'dateTime': '2018-11-25T09:22:00Z',
    'dateTimePub': None,
    'dataType': 'news',
    'sim': 0.615686297416687,
    'url': 'https://www.france24.com/en/20181125-france-yellow-vest-protesters-clash-police-champs-elysees-paris',
    'title': "'Yellow vest' protesters clash with police on Champs-Elysées in Paris - France 24",
    'body': 'Demonstrators wearing the yellow, high-visibility vests that symbolise their movement threw projectiles at police preventing them from moving along the famed shopping avenue, which was decked out in twinkling Christmas lights.\n\nThey also built barricades in some spots, and tore down traffic lights and street signs, creating riotous scenes reminiscent of France\'s 1968 civil unrest, or street insurrections in the mid-19th century immortalised in paintings a

In [72]:
bodies = [result['body'] for result in res_test['articles']['results']]

In [73]:
venezuela_in_bodies = False
count = 0
for body in bodies:
    if 'venezuela' in body or 'Venezuela' in body:
        venezuela_in_bodies = True
        count += 1
        print(body)

Venezuela's President Nicolas Maduro on Tuesday accused Washington of ordering a coup against his government and called for a "total revision" of ties with the US.

The latest claims by Caracas of a US coup plot came the day before a mass street protest announced by the opposition, and one day after a mutiny by some soldiers.

Maduro was specifically reacting to comments by US Vice President Mike Pence, who posted a video on Twitter branding Maduro "a dictator with no legitimate claim to power."

Referring to Wednesday's planned opposition rally, Pence added: "As the good people of Venezuela make your voices heard tomorrow, on behalf of the American people, we say: estamos con ustedes. We are with you."

Maduro responded during a radio and TV broadcast that "what the government of the United States did through Vice President Mike Pence, was to give orders to carry out a coup from the fascist state... which is unparalleled in the history of relations between the United States and Venezu

In [74]:
count 

9

In [112]:
stats = [(source, articles[source]['articles']['totalResults'], len(articles[source]['articles']['results'])) for source in articles if source != 'therussophile.org']

In [103]:
q = QueryArticlesIter(keywords=QueryItems.OR(protest_key_words),
        dateStart=date_start,
        dateEnd=date_end,
        sourceUri=er.getSourceUri('themoscowtimes.com'),
        lang=["eng"])
moscow_times_articles = []
for article in q.execQuery(er, sortBy = "rel", maxItems = 10000):
    moscow_times_articles.append(article)

In [119]:
moscow_times_articles[0]

{'uri': '1194049296',
 'lang': 'eng',
 'isDuplicate': False,
 'date': '2019-07-20',
 'time': '17:31:00',
 'dateTime': '2019-07-20T17:31:00Z',
 'dateTimePub': None,
 'dataType': 'news',
 'sim': 0.5686274766921997,
 'url': 'https://www.themoscowtimes.com/2019/07/20/a-pre-revolutionary-situation-more-than-20000-rally-in-moscow-for-free-elections-a66499',
 'title': "'A Pre-Revolutionary Situation': More Than 20,000 Rally in Moscow for Free Elections",
 'body': 'Russia\'s largest protest in years came after a week of demonstrations against the barring of opposition candidates from city council elections.\n\nRussian opposition leaders on Saturday led a protest in Moscow against the election commission\'s decision to bar a host of candidates from the ballot for city council elections this fall that turned into the largest demonstration in Russia in recent years.\n\nBilled as the culmination of a week of daily protests that kicked off last Sunday, the demonstration -- which was approved by the

In [114]:
sum([elt[1] for elt in stats])

54620